In [3]:
import pandas as pd
from sklearn.datasets import load_wine

In [4]:
wine = load_wine()

In [5]:
df = pd.DataFrame(wine.data, columns=wine.feature_names)

In [6]:
df.head(3)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0


In [7]:
y = wine.target

In [8]:
print(sum((df.proline.values * 10)%10))
print(sum((df.magnesium.values *10)%10))

0.0
0.0


In [9]:
df['proline'] = df.proline.astype(int)
df['magnesium'] = df.magnesium.astype(int)

In [10]:
df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [11]:
df.dtypes

alcohol                         float64
malic_acid                      float64
ash                             float64
alcalinity_of_ash               float64
magnesium                         int64
total_phenols                   float64
flavanoids                      float64
nonflavanoid_phenols            float64
proanthocyanins                 float64
color_intensity                 float64
hue                             float64
od280/od315_of_diluted_wines    float64
proline                           int64
dtype: object

In [12]:
set(y)

{0, 1, 2}

In [33]:
import warnings
import optuna.integration.lightgbm as lgb
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(df.values, y)

In [23]:
dtrain = lgb.Dataset(X_train, y_train)
dtest = lgb.Dataset(X_test, y_test)

In [37]:
params = {
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'num_class': 3,
}
best_params = dict()
tuning_history = list()

warnings.simplefilter("ignore")
model = lgb.train(
    params,
    dtrain,
    early_stopping_rounds=5,
    num_boost_round=30,
    valid_sets=[dtrain, dtest],
    feature_name=df.columns.tolist(),
    categorical_feature=['proline', 'magnesium']
    # best_params=best_params,
    # tuning_history=tuning_history
)
warnings.resetwarnings()

d_0's multi_logloss: 0.404492	valid_1's multi_logloss: 0.437267
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[10]	valid_0's multi_logloss: 0.368144	valid_1's multi_logloss: 0.404737
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[11]	valid_0's multi_logloss: 0.335943	valid_1's multi_logloss: 0.371096
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[12]	valid_0's multi_logloss: 0.307683	valid_1's multi_logloss: 0.342139
[LightGBM] [Warning] No further splits wit

In [38]:
model.params

/Users/wararaki/.pyenv/versions/3.8.5/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'objective': 'multiclass',
 'metric': 'multi_logloss',
 'num_class': 3,
 'feature_pre_filter': False,
 'lambda_l1': 0.0,
 'lambda_l2': 0.0,
 'num_leaves': 6,
 'feature_fraction': 0.8999999999999999,
 'bagging_fraction': 0.9995179917897894,
 'bagging_freq': 4,
 'min_child_samples': 10,
 'num_iterations': 30,
 'early_stopping_round': 5,
 'categorical_column': [4, 12]}

In [40]:
import numpy as np
y_preds = [np.argmax(row) for row in model.predict(X_test)]

/Users/wararaki/.pyenv/versions/3.8.5/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
from sklearn.metrics import accuracy_score

accuracy_score(y_preds, y_test)

/Users/wararaki/.pyenv/versions/3.8.5/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.9555555555555556